In [1]:
from pymatgen.core.structure import Structure
from pymatgen.analysis.ewald import EwaldSummation
import numpy as np
from tqdm import tqdm

In [2]:
def electrostatic_energy(structure, oxidation_states):
    structure.add_oxidation_state_by_element(oxidation_states)
    total_electrostatic_energy = EwaldSummation(structure, acc_factor=12.0).total_energy
    return total_electrostatic_energy

In [3]:
structure = Structure.from_file("ICSD_CollCode422259.cif")

for i, site in enumerate(structure):
    if not site.is_ordered:
        structure[i] = {"Li+": 1}
print("The composition after adjustments is %s." % structure.composition.reduced_formula)

The composition after adjustments is Li15La3Zr2O12.


In [4]:
tet_site = structure[0:24]
oct_site = structure[24:120]

In [5]:
def liindex(structure):
    li = []
    for i, site in enumerate(structure):
        if site.species_string == 'Li+':
            li.append(i)
    return li

def get_removed_li_structure(structure):
    global remove_t
    global remove_o
    ee = []
    for i in liindex(structure):
        s = structure.copy()
        s.remove_sites([i])
        e = electrostatic_energy(s, oxidation_states={"Li":1,"La":3,"Zr":4,"O":-2})
        ee.append(e)
    ss = structure.copy()
    ss.remove_sites([ee.index(min(ee))])
    if structure[ee.index(min(ee))] in tet_site:
        remove_t +=1
    else:
        remove_o +=1   
    return ss
def get_removed_li_structure_14(structure):
    ee = []
    for i in liindex(structure)[10:]:
        s = structure.copy()
        s.remove_sites([i])
        e = electrostatic_energy(s, oxidation_states={"Li":1,"La":3,"Zr":4,"O":-2})
        ee.append(e)
    ss = structure.copy()
    ss.remove_sites([ee.index(min(ee))])   
    return ss
def get_removed_li_structure_50(structure):
    ee = []
    for i in liindex(structure)[:-46]:
        s = structure.copy()
        s.remove_sites([i])
        e = electrostatic_energy(s, oxidation_states={"Li":1,"La":3,"Zr":4,"O":-2})
        ee.append(e)
    ss = structure.copy()
    ss.remove_sites([ee.index(min(ee))])   
    return ss

In [ ]:
remove_t = 0
remove_o = 0
sss = get_removed_li_structure(structure)   #移除1次
for i in tqdm(np.arange(0,63)):    #再移除63次
    if remove_t == 14:    #判断是否已经移除了14个四面体Li1
        sss = get_removed_li_structure_14(sss)
    elif remove_o == 50:  #判断是否已经移除了50个八面体Li2
        sss = get_removed_li_structure_50(sss)
    else:   
        sss = get_removed_li_structure(sss)

  0%|                                                    | 0/63 [00:00<?, ?it/s]

In [ ]:
sss.to(filename='cubic-Li7La3Zr2O12.cif')